# Squeeze-and-Excitation Network (SENet) in PyTorch and TensorFlow

For a more detailed breakdown of SENet, check out the [full tutorial on the blog](https://blog.paperspace.com/channel-attention-squeeze-and-excitation-networks/).

In [3]:
# Model Parameter and FLOP counter
# !pip install --upgrade git+https://github.com/sovrasov/flops-counter.pytorch.git

In [4]:
!nvidia-smi

Tue Sep 29 16:46:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P6000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   59C    P0   156W / 250W |   3119MiB / 24449MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
best_prec1 = 0
evaluate = True
import time
import torch
import torch.nn as nn
from ptflops import get_model_complexity_info
import torch.nn.functional as F
import torch.nn.init as init
import os
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import numpy as np
import math
from torch.nn.parameter import Parameter

def main():
    global best_prec1, evaluate

    __all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']


    def _weights_init(m):
        classname = m.__class__.__name__
        #print(classname)
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            init.kaiming_normal_(m.weight)


    class LambdaLayer(nn.Module):
        def __init__(self, lambd):
            super(LambdaLayer, self).__init__()
            self.lambd = lambd

        def forward(self, x):
            return self.lambd(x)

    ### Squeeze and Excitation Class definition
    class SE(nn.Module):
        def __init__(self, channel, reduction_ratio =16):
            super(SE, self).__init__()
            ### Global Average Pooling
            self.gap = nn.AdaptiveAvgPool2d(1)
            
            ### Fully Connected Multi-Layer Perceptron (FC-MLP)
            self.mlp = nn.Sequential(
                nn.Linear(channel, channel // reduction_ratio, bias=False),
                nn.ReLU(inplace=True),
                nn.Linear(channel // reduction_ratio, channel, bias=False),
                nn.Sigmoid()
            )

        def forward(self, x):
            b, c, _, _ = x.size()
            y = self.gap(x).view(b, c)
            y = self.mlp(y).view(b, c, 1, 1)
            return x * y.expand_as(x)


    class BasicBlock(nn.Module):
        expansion = 1

        def __init__(self, in_planes, planes, stride=1, option='A', use_se = True):
            super(BasicBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn2 = nn.BatchNorm2d(planes)
            self.use_se = use_se
            if self.use_se == True:
                self.se = SE(planes)

            self.shortcut = nn.Sequential()
            if stride != 1 or in_planes != planes:
                if option == 'A':
                    """
                    For CIFAR10 ResNet paper uses option A.
                    """
                    self.shortcut = LambdaLayer(lambda x:
                                                F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
                elif option == 'B':
                    self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion * planes)
                    )
            

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.bn2(self.conv2(out))
            out += self.shortcut(x)
            out = F.relu(out)
            if self.use_se == True:
                out = self.se(out)
            return out


    class ResNet(nn.Module):
        def __init__(self, block, num_blocks, num_classes=10):
            super(ResNet, self).__init__()
            self.in_planes = 16

            self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(16)
            self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
            self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
            self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
            self.linear = nn.Linear(64, num_classes)

            self.apply(_weights_init)

        def _make_layer(self, block, planes, num_blocks, stride):
            strides = [stride] + [1]*(num_blocks-1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, planes, stride))
                self.in_planes = planes * block.expansion

            return nn.Sequential(*layers)

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.layer3(out)
            out = F.avg_pool2d(out, out.size()[3])
            out = out.view(out.size(0), -1)
            out = self.linear(out)
            return out


    def resnet20():
        return ResNet(BasicBlock, [3, 3, 3])


    def resnet32():
        return ResNet(BasicBlock, [5, 5, 5])


    def resnet44():
        return ResNet(BasicBlock, [7, 7, 7])


    def resnet56():
        return ResNet(BasicBlock, [9, 9, 9])


    def resnet110():
        return ResNet(BasicBlock, [18, 18, 18])


    def resnet1202():
        return ResNet(BasicBlock, [200, 200, 200])

    model = resnet20()

    with torch.cuda.device(0):
      flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=True)
      print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
      print('{:<30}  {:<8}'.format('Number of parameters: ', params))

    model.cuda()

    cudnn.benchmark = True

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=128, shuffle=True,
        num_workers=4, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(model.parameters(), 0.1,
                                momentum=0.9,
                                weight_decay=5e-4)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                        milestones=[100, 150], last_epoch=0 - 1)


    for epoch in range(0, 200):

        # train for one epoch
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        train(train_loader, model, criterion, optimizer, epoch)
        lr_scheduler.step()

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        if epoch > 0 and epoch % 20 == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, is_best, filename=os.path.join('./', 'vanilla_checkpoint.th'))

        save_checkpoint({
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, filename=os.path.join('./', 'vanilla_model.th'))



def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = input.cuda()
        target_var = target

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 20 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))


def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()


            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res



In [6]:
if __name__ == '__main__':
    main()

ResNet(
  0.272 M, 100.000% Params, 2.01 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.0 M, 0.159% Params, 0.022 GMac, 1.079% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.080% MACs, 16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    0.014 M, 5.193% Params, 0.706 GMac, 35.114% MACs, 
    (0): BasicBlock(
      0.005 M, 1.731% Params, 0.235 GMac, 11.705% MACs, 
      (conv1): Conv2d(0.002 M, 0.848% Params, 0.116 GMac, 5.753% MACs, 16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.080% MACs, 16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(0.002 M, 0.848% Params, 0.116 GMac, 5.753% MACs, 16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.080% MACs, 16, eps=1e-05

Files already downloaded and verified
current lr 1.00000e-01
Epoch: [0][0/391]	Time 0.798 (0.798)	Data 0.240 (0.240)	Loss 2.3965 (2.3965)	Prec@1 13.281 (13.281)
Epoch: [0][20/391]	Time 0.064 (0.093)	Data 0.000 (0.012)	Loss 2.0916 (2.2359)	Prec@1 21.875 (14.174)
Epoch: [0][40/391]	Time 0.068 (0.076)	Data 0.000 (0.006)	Loss 1.9182 (2.1109)	Prec@1 20.312 (18.788)
Epoch: [0][60/391]	Time 0.063 (0.070)	Data 0.000 (0.004)	Loss 1.8620 (2.0432)	Prec@1 27.344 (20.761)
Epoch: [0][80/391]	Time 0.066 (0.067)	Data 0.000 (0.003)	Loss 1.8958 (1.9928)	Prec@1 21.094 (22.512)
Epoch: [0][100/391]	Time 0.055 (0.065)	Data 0.000 (0.003)	Loss 1.8057 (1.9547)	Prec@1 29.688 (24.080)
Epoch: [0][120/391]	Time 0.054 (0.063)	Data 0.000 (0.002)	Loss 1.7386 (1.9199)	Prec@1 30.469 (25.400)
Epoch: [0][140/391]	Time 0.057 (0.063)	Data 0.000 (0.002)	Loss 1.7962 (1.8953)	Prec@1 36.719 (26.585)
Epoch: [0][160/391]	Time 0.048 (0.062)	Data 0.000 (0.002)	Loss 1.5293 (1.8785)	Prec@1 40.625 (27.417)
Epoch: [0][180/391]	Time 0.

Epoch: [3][200/391]	Time 0.060 (0.060)	Data 0.000 (0.002)	Loss 0.9488 (0.9014)	Prec@1 68.750 (68.404)
Epoch: [3][220/391]	Time 0.050 (0.059)	Data 0.000 (0.002)	Loss 0.8739 (0.9006)	Prec@1 71.094 (68.375)
Epoch: [3][240/391]	Time 0.054 (0.059)	Data 0.000 (0.001)	Loss 0.7579 (0.8971)	Prec@1 73.438 (68.539)
Epoch: [3][260/391]	Time 0.052 (0.059)	Data 0.000 (0.001)	Loss 0.9361 (0.8983)	Prec@1 64.062 (68.502)
Epoch: [3][280/391]	Time 0.059 (0.059)	Data 0.000 (0.001)	Loss 0.9618 (0.8958)	Prec@1 67.969 (68.611)
Epoch: [3][300/391]	Time 0.059 (0.059)	Data 0.000 (0.001)	Loss 0.9098 (0.8946)	Prec@1 66.406 (68.651)
Epoch: [3][320/391]	Time 0.053 (0.059)	Data 0.000 (0.001)	Loss 0.9566 (0.8922)	Prec@1 67.188 (68.740)
Epoch: [3][340/391]	Time 0.065 (0.059)	Data 0.000 (0.001)	Loss 0.7931 (0.8895)	Prec@1 71.094 (68.871)
Epoch: [3][360/391]	Time 0.048 (0.059)	Data 0.000 (0.001)	Loss 0.7112 (0.8854)	Prec@1 77.344 (69.005)
Epoch: [3][380/391]	Time 0.052 (0.059)	Data 0.000 (0.001)	Loss 0.6907 (0.8809)	Pre

Test: [20/79]	Time 0.020 (0.026)	Loss 0.8012 (0.8714)	Prec@1 70.312 (69.234)
Test: [40/79]	Time 0.014 (0.020)	Loss 0.9485 (0.8407)	Prec@1 66.406 (71.113)
Test: [60/79]	Time 0.026 (0.019)	Loss 0.9097 (0.8440)	Prec@1 65.625 (70.761)
 * Prec@1 70.680
current lr 1.00000e-01
Epoch: [7][0/391]	Time 0.323 (0.323)	Data 0.249 (0.249)	Loss 0.5539 (0.5539)	Prec@1 78.906 (78.906)
Epoch: [7][20/391]	Time 0.064 (0.070)	Data 0.000 (0.012)	Loss 0.5711 (0.6499)	Prec@1 79.688 (77.753)
Epoch: [7][40/391]	Time 0.062 (0.064)	Data 0.000 (0.006)	Loss 0.6677 (0.6712)	Prec@1 78.125 (77.401)
Epoch: [7][60/391]	Time 0.050 (0.062)	Data 0.000 (0.004)	Loss 0.7037 (0.6695)	Prec@1 80.469 (77.177)
Epoch: [7][80/391]	Time 0.056 (0.061)	Data 0.000 (0.003)	Loss 0.5819 (0.6736)	Prec@1 77.344 (76.968)
Epoch: [7][100/391]	Time 0.048 (0.060)	Data 0.000 (0.003)	Loss 0.6271 (0.6735)	Prec@1 77.344 (76.903)
Epoch: [7][120/391]	Time 0.047 (0.059)	Data 0.000 (0.002)	Loss 0.7328 (0.6780)	Prec@1 75.781 (76.756)
Epoch: [7][140/391]	T

Epoch: [10][160/391]	Time 0.046 (0.054)	Data 0.000 (0.002)	Loss 0.4433 (0.6230)	Prec@1 84.375 (78.547)
Epoch: [10][180/391]	Time 0.048 (0.054)	Data 0.000 (0.002)	Loss 0.4874 (0.6257)	Prec@1 81.250 (78.479)
Epoch: [10][200/391]	Time 0.055 (0.054)	Data 0.000 (0.001)	Loss 0.6581 (0.6272)	Prec@1 80.469 (78.490)
Epoch: [10][220/391]	Time 0.064 (0.055)	Data 0.000 (0.001)	Loss 0.4831 (0.6274)	Prec@1 84.375 (78.468)
Epoch: [10][240/391]	Time 0.063 (0.055)	Data 0.000 (0.001)	Loss 0.7708 (0.6302)	Prec@1 75.781 (78.404)
Epoch: [10][260/391]	Time 0.061 (0.055)	Data 0.000 (0.001)	Loss 0.7403 (0.6299)	Prec@1 71.094 (78.397)
Epoch: [10][280/391]	Time 0.069 (0.055)	Data 0.000 (0.001)	Loss 0.6595 (0.6301)	Prec@1 76.562 (78.397)
Epoch: [10][300/391]	Time 0.060 (0.055)	Data 0.000 (0.001)	Loss 0.5780 (0.6314)	Prec@1 82.812 (78.395)
Epoch: [10][320/391]	Time 0.055 (0.055)	Data 0.000 (0.001)	Loss 0.6877 (0.6287)	Prec@1 75.000 (78.466)
Epoch: [10][340/391]	Time 0.051 (0.056)	Data 0.000 (0.001)	Loss 0.6325 (0

...

Epoch: [90][260/391]	Time 0.053 (0.055)	Data 0.000 (0.001)	Loss 0.4693 (0.4324)	Prec@1 82.031 (85.162)
Epoch: [90][280/391]	Time 0.046 (0.055)	Data 0.000 (0.001)	Loss 0.4077 (0.4332)	Prec@1 85.156 (85.162)
Epoch: [90][300/391]	Time 0.072 (0.055)	Data 0.000 (0.001)	Loss 0.4622 (0.4340)	Prec@1 81.250 (85.112)
Epoch: [90][320/391]	Time 0.068 (0.055)	Data 0.000 (0.001)	Loss 0.5346 (0.4354)	Prec@1 85.938 (85.069)
Epoch: [90][340/391]	Time 0.046 (0.056)	Data 0.000 (0.001)	Loss 0.3489 (0.4364)	Prec@1 88.281 (84.991)
Epoch: [90][360/391]	Time 0.055 (0.056)	Data 0.000 (0.001)	Loss 0.3659 (0.4353)	Prec@1 89.062 (85.050)
Epoch: [90][380/391]	Time 0.056 (0.056)	Data 0.000 (0.001)	Loss 0.4822 (0.4363)	Prec@1 89.062 (85.017)
Test: [0/79]	Time 0.232 (0.232)	Loss 0.8425 (0.8425)	Prec@1 77.344 (77.344)
Test: [20/79]	Time 0.014 (0.026)	Loss 0.9787 (0.8337)	Prec@1 76.562 (74.442)
Test: [40/79]	Time 0.014 (0.021)	Loss 1.0252 (0.8209)	Prec@1 71.875 (75.019)
Test: [60/79]	Time 0.014 (0.019)	Loss 0.7100 (0.8

 * Prec@1 77.360
current lr 1.00000e-01
Epoch: [94][0/391]	Time 0.369 (0.369)	Data 0.312 (0.312)	Loss 0.3547 (0.3547)	Prec@1 83.594 (83.594)
Epoch: [94][20/391]	Time 0.049 (0.072)	Data 0.000 (0.015)	Loss 0.4455 (0.4112)	Prec@1 89.844 (85.342)
Epoch: [94][40/391]	Time 0.048 (0.065)	Data 0.000 (0.008)	Loss 0.5533 (0.4166)	Prec@1 81.250 (85.366)
Epoch: [94][60/391]	Time 0.056 (0.062)	Data 0.000 (0.005)	Loss 0.4584 (0.4202)	Prec@1 84.375 (85.374)
Epoch: [94][80/391]	Time 0.062 (0.061)	Data 0.000 (0.004)	Loss 0.4169 (0.4199)	Prec@1 87.500 (85.619)
Epoch: [94][100/391]	Time 0.064 (0.060)	Data 0.000 (0.003)	Loss 0.5253 (0.4268)	Prec@1 84.375 (85.466)
Epoch: [94][120/391]	Time 0.057 (0.060)	Data 0.000 (0.003)	Loss 0.6281 (0.4318)	Prec@1 81.250 (85.324)
Epoch: [94][140/391]	Time 0.059 (0.059)	Data 0.000 (0.003)	Loss 0.4894 (0.4298)	Prec@1 86.719 (85.278)
Epoch: [94][160/391]	Time 0.057 (0.059)	Data 0.000 (0.002)	Loss 0.3359 (0.4277)	Prec@1 89.844 (85.321)
Epoch: [94][180/391]	Time 0.061 (0.059)

Epoch: [97][200/391]	Time 0.065 (0.055)	Data 0.000 (0.001)	Loss 0.4062 (0.4206)	Prec@1 85.938 (85.479)
Epoch: [97][220/391]	Time 0.062 (0.055)	Data 0.000 (0.001)	Loss 0.4421 (0.4222)	Prec@1 88.281 (85.453)
Epoch: [97][240/391]	Time 0.059 (0.056)	Data 0.000 (0.001)	Loss 0.3986 (0.4244)	Prec@1 85.156 (85.403)
Epoch: [97][260/391]	Time 0.061 (0.056)	Data 0.000 (0.001)	Loss 0.6039 (0.4258)	Prec@1 77.344 (85.387)
Epoch: [97][280/391]	Time 0.061 (0.056)	Data 0.000 (0.001)	Loss 0.4370 (0.4286)	Prec@1 85.156 (85.292)
Epoch: [97][300/391]	Time 0.063 (0.056)	Data 0.000 (0.001)	Loss 0.5042 (0.4286)	Prec@1 83.594 (85.294)
Epoch: [97][320/391]	Time 0.059 (0.056)	Data 0.000 (0.001)	Loss 0.5756 (0.4289)	Prec@1 82.031 (85.295)
Epoch: [97][340/391]	Time 0.044 (0.056)	Data 0.000 (0.001)	Loss 0.5089 (0.4295)	Prec@1 85.156 (85.321)
Epoch: [97][360/391]	Time 0.049 (0.055)	Data 0.000 (0.001)	Loss 0.4986 (0.4306)	Prec@1 82.031 (85.264)
Epoch: [97][380/391]	Time 0.039 (0.054)	Data 0.000 (0.001)	Loss 0.4233 (0

Test: [0/79]	Time 0.221 (0.221)	Loss 0.2099 (0.2099)	Prec@1 91.406 (91.406)
Test: [20/79]	Time 0.011 (0.024)	Loss 0.2564 (0.3006)	Prec@1 89.844 (89.583)
Test: [40/79]	Time 0.014 (0.019)	Loss 0.3860 (0.2996)	Prec@1 89.062 (89.653)
Test: [60/79]	Time 0.016 (0.017)	Loss 0.2392 (0.2935)	Prec@1 92.188 (89.780)
 * Prec@1 89.920
current lr 1.00000e-02
Epoch: [101][0/391]	Time 0.354 (0.354)	Data 0.295 (0.295)	Loss 0.4191 (0.4191)	Prec@1 83.594 (83.594)
Epoch: [101][20/391]	Time 0.032 (0.058)	Data 0.000 (0.014)	Loss 0.2127 (0.2556)	Prec@1 90.625 (90.960)
Epoch: [101][40/391]	Time 0.038 (0.050)	Data 0.000 (0.007)	Loss 0.3280 (0.2457)	Prec@1 91.406 (91.578)
Epoch: [101][60/391]	Time 0.041 (0.047)	Data 0.000 (0.005)	Loss 0.2423 (0.2503)	Prec@1 91.406 (91.534)
Epoch: [101][80/391]	Time 0.045 (0.045)	Data 0.002 (0.004)	Loss 0.2843 (0.2509)	Prec@1 91.406 (91.474)
Epoch: [101][100/391]	Time 0.036 (0.045)	Data 0.000 (0.003)	Loss 0.2695 (0.2502)	Prec@1 90.625 (91.491)
Epoch: [101][120/391]	Time 0.042 (0

KeyboardInterrupt: 